In [6]:
# Install dependencies
!pip install gradio openai fpdf

In [7]:
import gradio as gr
import openai
import re
from fpdf import FPDF
import tempfile
import os

In [16]:
openai.api_key = "OPENAI_API_KEY"

In [17]:
VALID_COUNTRY_CODES = {
    "+1", "+7", "+20", "+27", "+30", "+31", "+32", "+33", "+34", "+36", "+39", "+40", "+41", "+43", "+44", "+45", "+46", "+47", "+48", "+49",
    "+51", "+52", "+53", "+54", "+55", "+56", "+57", "+58", "+60", "+61", "+62", "+63", "+64", "+65", "+66", "+81", "+82", "+84", "+86", "+90",
    "+91", "+92", "+93", "+94", "+95", "+98", "+211", "+212", "+213", "+216", "+218", "+220", "+221", "+222", "+223", "+224", "+225", "+226",
    "+227", "+228", "+229", "+230", "+231", "+232", "+233", "+234", "+235", "+236", "+237", "+238", "+239", "+240", "+241", "+242", "+243",
    "+244", "+245", "+246", "+248", "+249", "+250", "+251", "+252", "+253", "+254", "+255", "+256", "+257", "+258", "+260", "+261", "+262",
    "+263", "+264", "+265", "+266", "+267", "+268", "+269", "+290", "+291", "+297", "+298", "+299", "+350", "+351", "+352", "+353", "+354",
    "+355", "+356", "+357", "+358", "+359", "+370", "+371", "+372", "+373", "+374", "+375", "+376", "+377", "+378", "+380", "+381", "+382",
    "+383", "+385", "+386", "+387", "+389", "+420", "+421", "+423", "+500", "+501", "+502", "+503", "+504", "+505", "+506", "+507", "+508",
    "+509", "+590", "+591", "+592", "+593", "+594", "+595", "+596", "+597", "+598", "+599", "+670", "+672", "+673", "+674", "+675", "+676",
    "+677", "+678", "+679", "+680", "+681", "+682", "+683", "+685", "+686", "+687", "+688", "+689", "+690", "+691", "+692", "+850", "+852",
    "+853", "+855", "+856", "+870", "+871", "+872", "+873", "+874", "+878", "+880", "+881", "+882", "+883", "+886", "+960", "+961", "+962",
    "+963", "+964", "+965", "+966", "+967", "+968", "+970", "+971", "+972", "+973", "+974", "+975", "+976", "+977", "+992", "+993", "+994",
    "+995", "+996", "+998"
}

In [18]:
QUESTIONS = [
    {"label": "👤 What is your full name?", "key": "name"},
    {"label": "📧 What is your email address?", "key": "email"},
    {"label": "🌐 Enter your country code (e.g., +91, +1, +44):", "key": "country_code"},
    {"label": "📱 Enter your phone number (10 digits, no country code):", "key": "phone"},
    {"label": "💼 How many years of professional experience do you have?", "key": "experience"},
    {"label": "🧑‍💻 For which role(s) have you worked during these years?", "key": "roles"},
    {"label": "📂 Can you briefly describe one or more projects you worked on in this role? Please provide a link (GitHub, portfolio, etc.) if available.", "key": "projects"},
    {"label": "🎯 Which position(s) are you applying for?", "key": "position"},
    {"label": "📍 Where are you currently located?", "key": "location"},
    {"label": "🛠️ Please list your tech stack (e.g., Python, Django, React, C++, Node.js, C#):", "key": "tech_stack"},
    # Python libraries will be inserted dynamically if needed
]

PYTHON_LIBS_QUESTION = {
    "label": (
        "🐍 You mentioned Python. Which Python data science or machine learning libraries have you used? "
        "(e.g., Pandas, NumPy, TensorFlow, PyTorch, SciPy, Keras, Scikit-learn, Matplotlib, Seaborn, OpenCV, Plotly, XGBoost, LightGBM, Theano, NLTK, SpaCy, etc.)"
    ),
    "key": "python_libraries"
}

In [19]:
def validate_input(answer, step, data, python_libs_needed):
    if step == 0:  # Name
        if not re.fullmatch(r"[A-Za-z ]+", answer.strip()):
            return False, "Name must contain only alphabets and spaces. Please enter your full name."
    elif step == 1:  # Email
        if not re.fullmatch(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.com", answer.strip()):
            return False, "Email must be valid and end with .com. Please enter a valid email address."
    elif step == 2:  # Country code
        if answer.strip() not in VALID_COUNTRY_CODES:
            return False, "Invalid country code. Please enter a valid country code (e.g., +91, +1, +44)."
    elif step == 3:  # Phone number (10 digits)
        if not re.fullmatch(r"\d{10}$", answer.strip()):
            return False, "Phone number must be exactly 10 digits (no country code)."
    elif step == 4:  # Experience
        if not re.fullmatch(r"\d+", answer.strip()):
            return False, "Experience must be a number (in years). Please enter your years of experience."
    elif step == 5:  # Roles
        if not answer.strip():
            return False, "Please specify the role(s) you have worked in."
    elif step == 6:  # Projects
        if not answer.strip():
            return False, "Please describe at least one project and provide a link if available."
    elif step == 7:  # Position
        if not re.fullmatch(r"[A-Za-z ]+", answer.strip()):
            return False, "Position must contain only alphabets and spaces. Please enter the position(s)."
    elif step == 8:  # Location
        if not re.fullmatch(r"[A-Za-z ]+", answer.strip()):
            return False, "Location must contain only alphabets and spaces. Please enter your location."
    elif step == 9:  # Tech Stack
        if not re.fullmatch(r"[A-Za-z0-9, .#+-]+", answer.strip()):
            return False, "Tech stack must contain only alphabets, numbers, spaces, commas, periods, plus, or hash (e.g., Python, C++, C#, Node.js)."
    elif python_libs_needed and step == 10:  # Python Libraries
        if not answer.strip():
            return False, "Please specify at least one Python data science or ML library."
    return True, ""


In [20]:
def generate_technical_questions(tech_stack):
    prompt = (
        f"Generate 3 technical interview questions for each of the following technologies: {tech_stack}. "
        "Format as numbered questions."
    )
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert technical interviewer."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [21]:
def create_pdf(data):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, "TalentScout Candidate Summary", ln=True, align="C")
    pdf.ln(8)
    for key, value in data.items():
        label = key.replace("_", " ").title()
        pdf.multi_cell(0, 10, f"{label}:\n{value}\n", align="L")
    temp = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    pdf.output(temp.name)
    temp.close()
    return temp.name

In [22]:
def chatbot(user_message, chat_history, step, data, python_libs_needed, ended, pdf_path):
    user_message_clean = user_message.strip().lower() if user_message else ""
    if user_message_clean in ["end", "exit", "quit", "bye"]:
        chat_history.append({"role": "user", "content": user_message})
        chat_history.append({"role": "assistant", "content": "🌈 Thank you for your time!\n\nType **start** to begin again or download your summary PDF below."})
        return "", chat_history, step, data, python_libs_needed, True, pdf_path
    if user_message_clean == "start":
        chat_history.append({"role": "user", "content": user_message})
        chat_history.append({"role": "assistant", "content": QUESTIONS[0]["label"]})
        return "", chat_history, 0, {}, False, False, None
    if ended:
        chat_history.append({"role": "user", "content": user_message})
        chat_history.append({"role": "assistant", "content": "Type **start** to begin the screening process again or download your PDF below."})
        return "", chat_history, step, data, python_libs_needed, ended, pdf_path

    # If we're at the Python libraries question
    if step == 10 and python_libs_needed:
        is_valid, error_msg = validate_input(user_message, step, data, python_libs_needed)
        if not is_valid:
            chat_history.append({"role": "user", "content": user_message})
            chat_history.append({"role": "assistant", "content": f"❗ {error_msg} {PYTHON_LIBS_QUESTION['label']}"})
            return "", chat_history, step, data, python_libs_needed, ended, pdf_path
        data[PYTHON_LIBS_QUESTION["key"]] = user_message
        chat_history.append({"role": "user", "content": user_message})
        # All done, generate tech questions and PDF
        tech_stack = data.get("tech_stack", "")
        try:
            tech_qs = generate_technical_questions(tech_stack)
            chat_history.append({"role": "assistant", "content": "🎯 **Thank you! Here are your technical questions:**\n\n" + tech_qs})
        except Exception as e:
            chat_history.append({"role": "assistant", "content": f"❗ Sorry, there was an error generating technical questions: {e}."})
        pdf_file = create_pdf(data)
        chat_history.append({"role": "assistant", "content": "📄 Click the button below to download your candidate summary as a PDF."})
        return "", chat_history, step+1, data, False, True, pdf_file

    # Normal flow
    is_valid, error_msg = validate_input(user_message, step, data, python_libs_needed)
    if not is_valid:
        chat_history.append({"role": "user", "content": user_message})
        label = PYTHON_LIBS_QUESTION["label"] if (python_libs_needed and step == 10) else QUESTIONS[step]["label"]
        chat_history.append({"role": "assistant", "content": f"❗ {error_msg} {label}"})
        return "", chat_history, step, data, python_libs_needed, ended, pdf_path

    # Save answer and advance
    if step < len(QUESTIONS):
        data[QUESTIONS[step]["key"]] = user_message
        chat_history.append({"role": "user", "content": user_message})
        step += 1
        # After tech stack, check for Python
        if step == 10:
            tech_stack = data.get("tech_stack", "")
            if "python" in tech_stack.lower():
                chat_history.append({"role": "assistant", "content": PYTHON_LIBS_QUESTION["label"]})
                return "", chat_history, step, data, True, ended, pdf_path
        if step < len(QUESTIONS):
            next_q = QUESTIONS[step]["label"]
            chat_history.append({"role": "assistant", "content": next_q})
            return "", chat_history, step, data, python_libs_needed, ended, pdf_path
        else:
            # All done, generate tech questions and PDF
            tech_stack = data.get("tech_stack", "")
            try:
                tech_qs = generate_technical_questions(tech_stack)
                chat_history.append({"role": "assistant", "content": "🎯 **Thank you! Here are your technical questions:**\n\n" + tech_qs})
            except Exception as e:
                chat_history.append({"role": "assistant", "content": f"❗ Sorry, there was an error generating technical questions: {e}."})
            pdf_file = create_pdf(data)
            chat_history.append({"role": "assistant", "content": "📄 Click the button below to download your candidate summary as a PDF."})
            return "", chat_history, step, data, False, True, pdf_file
    else:
        chat_history.append({"role": "user", "content": user_message})
        chat_history.append({"role": "assistant", "content": "🎉 **Thank you for completing the initial screening!**\n\nOur team will review your responses and contact you for next steps."})
        return "", chat_history, 0, {}, False, ended, pdf_path

In [23]:
def download_pdf(pdf_path):
    if pdf_path and os.path.exists(pdf_path):
        return pdf_path
    return None

with gr.Blocks(theme=gr.themes.Glass(), css="""
body {
    background: linear-gradient(135deg, #e0c3fc 0%, #8ec5fc 100%) !important;
}
""") as demo:
    gr.HTML(
        """
        <div style='display:flex;align-items:center;gap:1em;'>
            <img id="logo" src="https://cdn-icons-png.flaticon.com/512/4712/4712027.png" height="60">
            <div>
                <h1 style='margin-bottom:0;color:#6366f1;'>TalentScout Hiring Assistant 🤖</h1>
                <p style='margin-top:0;font-size:1.1em;color:#6366f1;'>Your AI-powered candidate screener</p>
            </div>
        </div>
        """)
    gr.Markdown(
        """
<span style="color:#6366f1;font-weight:bold;">Welcome!</span>
I will guide you through the initial screening process.<br>
Type <span style="color:#16a34a;font-weight:bold;">end</span> at any time to finish and <span style="color:#6366f1;font-weight:bold;">start</span> to begin again.
        """
    )
    chatbot_ui = gr.Chatbot(type="messages", label="TalentScout Chat", avatar_images=("https://cdn-icons-png.flaticon.com/512/4712/4712027.png", None))
    msg = gr.Textbox(placeholder="Type your answer here...", label="Your Answer")
    pdf_btn = gr.Button("Download PDF", visible=False)
    pdf_file = gr.File(label="Your Candidate Summary PDF", visible=False)
    clear = gr.ClearButton([msg, chatbot_ui])
    state_step = gr.State(0)
    state_data = gr.State({})
    state_python_libs_needed = gr.State(False)
    state_ended = gr.State(False)
    state_pdf_path = gr.State(None)

    def initial_history():
        return [{"role": "assistant", "content": QUESTIONS[0]["label"]}]

    def update_pdf_btn(pdf_path, ended):
        return gr.update(visible=bool(pdf_path and ended))

    def update_pdf_file(pdf_path):
        return gr.update(value=pdf_path, visible=True)

    msg.submit(
        chatbot,
        [msg, chatbot_ui, state_step, state_data, state_python_libs_needed, state_ended, state_pdf_path],
        [msg, chatbot_ui, state_step, state_data, state_python_libs_needed, state_ended, state_pdf_path],
        queue=False
    )
    demo.load(lambda: ("", initial_history(), 0, {}, False, False, None), outputs=[msg, chatbot_ui, state_step, state_data, state_python_libs_needed, state_ended, state_pdf_path])
    pdf_btn.click(download_pdf, [state_pdf_path], [pdf_file])
    state_pdf_path.change(update_pdf_btn, [state_pdf_path, state_ended], [pdf_btn])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5946344b2b587d3f1e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
